In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification, RobertaConfig
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Load dataset
df = pd.read_csv("/content/fake reviews dataset (5).csv")

In [ ]:
# Preprocessing
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
    tokens = [word.lower() for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    tokens = [stemmer.stem(word) for word in tokens]  # Stemming
    return ' '.join(tokens)

df['cleaned_text'] = df['text_'].apply(preprocess_text)

In [ ]:
# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
X = tfidf.fit_transform(df['cleaned_text']).toarray()
X_normalized = normalize(X)

In [ ]:
# Classification: Logistic Regression and SVM
X_class = tfidf.transform(df['cleaned_text']).toarray()
y_class = df['label']
X_train, X_test, y_train, y_test = train_test_split(X_class, y_class, test_size=0.3, random_state=42)

In [ ]:
# LSTM Model
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['cleaned_text'])
X_seq = tokenizer.texts_to_sequences(df['cleaned_text'])
X_padded = pad_sequences(X_seq, maxlen=200)

X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(X_padded, y_class, test_size=0.3, random_state=42)
le = LabelEncoder()
y_train_dl = le.fit_transform(y_train_dl)
y_test_dl = le.transform(y_test_dl)

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_dl, y_train_dl, epochs=5, batch_size=64, validation_split=0.2)
lstm_results = model.evaluate(X_test_dl, y_test_dl)
print("LSTM Results:\n", lstm_results)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
354/354 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - accuracy: 0.7474 - loss: 0.5122 - val_accuracy: 0.8977 - val_loss: 0.2407
Epoch 2/5
354/354 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9200 - loss: 0.2040 - val_accuracy: 0.9096 - val_loss: 0.2199
Epoch 3/5
354/354 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9427 - loss: 0.1517 - val_accuracy: 0.9094 - val_loss: 0.2215
Epoch 4/5
354/354 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.9552 - loss: 0.1145 - val_accuracy: 0.9164 - val_loss: 0.2210
Epoch 5/5
354/354 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9664 - loss: 0.0938 - val_accuracy: 0.9140 - val_loss: 0.2571
380/380 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9119 - loss: 0.2746
LSTM Results:
 [0.2740212082862854, 0.9079967141151428]


In [ ]:
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification, RobertaConfig
from sklearn.preprocessing import LabelEncoder # Import LabelEncoder

tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer_roberta(list(df['cleaned_text']), truncation=True, padding=True, max_length=200, return_tensors="tf")
X_train_roberta, X_test_roberta, y_train_roberta, y_test_roberta = train_test_split(
    np.array(train_encodings['input_ids']),
    y_class,
    test_size=0.3,
    random_state=42
)
le = LabelEncoder()
y_train_roberta = le.fit_transform(y_train_roberta)
y_test_roberta = le.transform(y_test_roberta)
config = RobertaConfig.from_pretrained('roberta-base', num_labels=2)
model_roberta = TFRobertaForSequenceClassification.from_pretrained('roberta-base', config=config)
model_roberta.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model_roberta.fit(X_train_roberta, y_train_roberta, epochs=3, batch_size=8) # Now using numerical labels
roberta_results = model_roberta.evaluate(X_test_roberta, y_test_roberta) # Now using numerical labels
print("RoBERTa Results:\n", roberta_results)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/3
3538/3538 [==============================] - 1288s 346ms/step - loss: 0.7050 - accuracy: 0.4983
Epoch 2/3
3538/3538 [==============================] - 1215s 344ms/step - loss: 0.6980 - accuracy: 0.4962
Epoch 3/3
380/380 [==============================] - 154s 397ms/step - loss: 0.6971 - accuracy: 0.5001
RoBERTa Results:
 [0.6970633864402771, 0.5000824332237244]
